In [1]:
import os
if "x_perceiver" not in os.listdir():
    os.chdir("/home/kh701/pycharm/healnet/")
import torch
from torch import nn
import multiprocessing
import torchvision
import numpy as np
import torchvision.transforms as transforms
import einops
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from healnet.models.explainer import Explainer
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

from healnet.utils import Config, flatten_config
from healnet.etl import TCGADataset
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

    
%reload_ext autoreload
%autoreload 2

## Import data

In [2]:
# get dataloaders
config = Config("config/main_gpu.yml").read()
config = flatten_config(config) # TODO - refactor to other 

blca = TCGADataset(
    dataset="blca", 
    config=config, 
    level=2, 
    sources=["omic"]
)

brca = TCGADataset(
    dataset="brca", 
    config=config, 
    level=2, 
    sources=["omic"]
)



Filled 0 missing values with mean
Missing values per feature: 
 Series([], dtype: int64)
Slides available: 436
Omic available: 437
Overlap: 436
Filtering out 1 samples for which there are no omic data available
Dataloader initialised for blca dataset
Dataset: BLCA
Molecular data shape: (436, 2191)
Molecular/Slide match: 436/436
Slide level count: 4
Slide level dimensions: ((79968, 79653), (19992, 19913), (4998, 4978), (2499, 2489))
Slide resize dimensions: w: 1024, h: 1024
Sources selected: ['omic']
Censored share: 0.539
Survival_bin_sizes: {0: 72, 1: 83, 2: 109, 3: 172}
Filled 0 missing values with mean
Missing values per feature: 
 Series([], dtype: int64)
Slides available: 1019
Omic available: 1022
Overlap: 1019
Filtering out 3 samples for which there are no omic data available
Dataloader initialised for brca dataset
Dataset: BRCA
Molecular data shape: (1019, 2922)
Molecular/Slide match: 1019/1019
Slide level count: 3
Slide level dimensions: ((35855, 34985), (8963, 8746), (2240, 218

In [3]:
# get tabular data
blca_loader = DataLoader(
    blca, 
    batch_size=1, 
    shuffle=True, 
    num_workers=multiprocessing.cpu_count()-1
)
[blca_omic], censorship, event_time, y_disc = next(iter(blca_loader))

In [4]:
blca_omic.shape

torch.Size([1, 1, 2183])

## Tabular self-supervised pre-training

To start with, we want to build and encoder-decoder model which trains a cross-attention unit as the encoder, which can later on be deployed in the iterative model. We then want to benchmark the performance with pan-cancer pre-training vs. without pre-training. 

In [39]:
from healnet.models.healnet import Attention, PreNorm

class AttentionEncoder(nn.Module): 
    """
    Simple encoder that uses fourier encoding, pre-norm and cross-attention to encode the input features into a latent array 
    of size (num_latents x latent_dim). Takes in both the input tensors as well as a randomly initialised latent 
    array as the input. 
    """
    def __init__(self, 
                 input_channels: int,
                 latent: torch.Tensor, 
                 input_axis: int = 1, 
                 attn_dropout: float = 0.1,
                 num_heads: int = 4, 
                 num_freq_bands: int=8, 
                 ):    
        super().__init__()
        
        self.input_channels = input_channels
        self.input_axis = input_axis
        self.attn_dropout = attn_dropout
        self.num_heads = num_heads
        
        
        # fourier_channels = (input_axis * ((num_freq_bands * 2) + 1))
        # input_dim = fourier_channels + input_channels
        input_dim = input_channels
                
        latent_dim = latent.shape[-1] # required for PreNorm layer
        # simple single attention unit
        print(latent_dim)
        print(latent.shape)
        print(input_dim)
        enc = PreNorm(latent_dim, Attention(latent_dim, input_dim, heads=num_heads, dim_head=num_heads, dropout=attn_dropout), context_dim=input_dim)
        
        self.layers = nn.ModuleList([enc])
        
    def forward(self, latent: torch.Tensor, context: torch.Tensor):
        """
        Note: context is the data, x is the latent
        Args:
            latent: 
            context: 

        Returns:

        """
        for layer in self.layers:
            latent = layer(x=latent, context=context)
            
        print(latent.shape)
        print(latent.dtype)
        return latent


The decoder often needs to be different depending on the modality, so let's implement modality-specific decoders while trying to have a relatively general-purpose encoder that we can plug into the pipeline.

Note that we may change this later down the line. 

In [6]:
class TabularDecoder(nn.Module):
    """
    Decoder suited for tabular data. We use the following: 
    - Skip connections: faster and more stable training
    - Batch normalisation: stabilises the activations and speeds up training
    - Activation: Output layer to map back to output dimensions, corresponding to the original data dims
    Tries to reconstruct the original input given the latent
    """
    def __init__(self, latent_dim: int, output_dim: int):
        super(TabularDecoder, self).__init__()
        
        layers = []
        hidden_dims = [128, 256] # may refactor as hyperparameter later
        
        for hidden_dim in hidden_dims: 
            layers.extend([
                nn.Linear(latent_dim, hidden_dim), 
                nn.LeakyReLU(), 
                nn.BatchNorm1d(hidden_dim), 
                nn.Dropout(0.5)
            ])
            
            latent_dim = hidden_dim # update for next layer
        
        # final layer to reconstruct output
        layers.append(nn.Linear(latent_dim, output_dim))
        
        self.decode = nn.Sequential(*layers)
        
    def forward(self, latent: torch.Tensor):
        return self.decode(latent)
    
    
        
    

Finally, putting it all together in the encoder-decoder model


In [50]:
from typing import *

class TabPretrainer(nn.Module): 
    """
    Encoder-decoder model for pre-training tabular data.
    # TODO - refactor abstract base class for initialisations 
    """
    def __init__(self,
                 input_channels: int,
                 latent_shape: List[int],
                 input_axis: int = 1,
                 attn_dropout: float = 0.1,
                 num_heads: int = 4,
                 num_freq_bands: int=8,
                 ):
        super().__init__()
        self.input_channels = input_channels
        self.input_axis = input_axis
        self.num_latents, self.latent_dim = latent_shape
        self.attn_dropout = attn_dropout
        self.num_heads = num_heads
        self.num_freq_bands = num_freq_bands
        
        # randomly initialise latent
        self.latent = nn.Parameter(torch.randn(self.num_latents, self.latent_dim))
        
        # encoder
        self.encoder = AttentionEncoder(
            input_channels=input_channels, 
            latent=self.latent, 
            input_axis=self.input_axis, 
            attn_dropout=attn_dropout, 
            num_heads=num_heads, 
            num_freq_bands=num_freq_bands
        )
        
        # decoder
        self.decoder = TabularDecoder(
            latent_dim=self.latent_dim, 
            output_dim=input_channels
        )
        
    def forward(self, x: torch.Tensor):
        # get batch dim
        b = x.shape[0]
        
        # expand latent to batch size
        l = einops.repeat(self.latent, "n d -> b n d", b=b)
        
        print(self.latent.shape)
        # encode
        # self.latent = self.encoder(latent=l, context=x) + x
        self.latent.data = self.encoder(latent=l, context=x).data + l.data
        
        print(self.latent.shape)
        # decode, reconstructed x
        rec_x = self.decoder(self.latent)
        return rec_x
    
    def get_latent(self):
        return self.latent

Next, we need to think about tabular loss functions. Here, we can explore both reconstruction losses and contrastive losses. 

In [8]:
class TabularLoss(nn.Module):
    """
    Reconstruction loss functions for tabular data. We use two types which are commonly used with continuous data: 
    - Mean squared error
    - Constrastive loss, measured as cosine distance between the original and reconstructed data
    We seek to minimise both objectives.
    """
    def __init__(self,
                 method: str = "mse",
                 reduction: str = "mean",
                 ):
        super().__init__()
        assert method in ["mse", "contrastive"], "Loss type not recognised"
        self.loss_type = method
        self.reduction = reduction
        
        if method == "mse":
            self.loss = nn.MSELoss(reduction=reduction)
        elif method == "contrastive":
            self.loss = nn.CosineEmbeddingLoss(reduction=reduction)
            
    def __call__(self, x: torch.Tensor, rec_x: torch.Tensor):
        return self.loss(x, rec_x)
    

Finally, we write a pre-training loop that we can use for pre-training across cancer sites. 

In [9]:
# get overlap between omic columns
col1 = blca.omic_df.columns
col2 = brca.omic_df.columns
print(len(col1), len(col2), len(set(col1).intersection(col2)))

2191 2922 1758


In [51]:
from tqdm import tqdm

def pretrain_loop(
        data: TCGADataset,
        batch_size: int, 
    ):
    
    loader = DataLoader(
        data, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=multiprocessing.cpu_count()-1
    )
    [omic_sample], _, _, _ = next(iter(loader)) 
    
    model = TabPretrainer(
        input_channels=omic_sample.shape[-1], 
        input_axis=1, 
        latent_shape=[256, 32], 
        attn_dropout=0.1, 
        num_heads=4,
        num_freq_bands=8
    )
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    loss_fn = TabularLoss(method="mse")
    
    for epoch in tqdm(range(10)):
        for idx, batch in enumerate(loader):
            [omic], censorship, event_time, y_disc = batch
            print(omic.shape)
            # omic = einops.rearranged(omic, "b d c -> b c d")
            # omic = einops.rearrange(omic, "b () c -> b c")
            # omic = einops.reduce(omic, "b d c -> b c", "mean")
            # return reconstructed omic data
            rec_omic = model(omic)
            loss = loss_fn(omic, rec_omic)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            # print every 10th batch
            if idx % 10 == 0:
                print(loss)
            
    
pretrain_loop(
    data=blca, 
    batch_size=2)
    

32
torch.Size([256, 32])
2183


  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([2, 1, 2183])
torch.Size([256, 32])
torch.Size([2, 256, 32])
torch.Size([2, 1, 2183])
torch.Size([2, 256, 32])
torch.float32
torch.Size([2, 256, 32])


  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: running_mean should contain 256 elements not 128